In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [5]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'lbfgs'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [9]:
# Initialize MLflow
mlflow.set_experiment("deployment model")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2026/01/24 13:16:41 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/01/24 13:16:41 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/01/24 13:16:41 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/01/24 13:16:41 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/01/24 13:16:41 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/01/24 13:16:41 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/01/24 13:16:42 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/24 13:16:42 INFO mlflow.store.db.utils: Updating database tables
2026/01/24 13:16:42 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/24 13:16:42 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/24 13:16:42 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/24 13:16:42 INFO alembic.runtime

🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/3/runs/e0b264611d854a7b9a553632efc344b5
🧪 View experiment at: http://localhost:5000/#/experiments/3


2026/01/24 13:16:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/3/runs/3463d201f1fb4406bbd8ede2cb4e6d1e
🧪 View experiment at: http://localhost:5000/#/experiments/3


2026/01/24 13:16:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/3/runs/800bebef10fc4c8f9c6e4086d7f114c9
🧪 View experiment at: http://localhost:5000/#/experiments/3
🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/3/runs/e6f5f1ea5b9d41a9a006806fd3f76896
🧪 View experiment at: http://localhost:5000/#/experiments/3


### Register the Model

In [16]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2026/01/24 13:54:40 WARNING mlflow.tracking._model_registry.fluent: Run with id e6f5f1ea5b9d41a9a006806fd3f76896 has no artifacts at artifact path 'model', registering model based on models:/m-4640afd9724a4a56a57a600aba6a0209 instead
2026/01/24 13:54:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 3


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/3/runs/e6f5f1ea5b9d41a9a006806fd3f76896
🧪 View experiment at: http://localhost:5000/#/experiments/3


Created version '3' of model 'XGB-Smote'.


### Load the Model

In [14]:
model_name = 'XGB-Smote'
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production server

In [15]:
current_model_uri = f"models:/{model_name}@appserver"
production_model_name = "finalproduction"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Registered model 'finalproduction' already exists. Creating a new version of this model...
Copied version '1' of model 'XGB-Smote' to version '2' of model 'finalproduction'.


<ModelVersion: aliases=[], creation_timestamp=1769242949264, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1769242949264, metrics=None, model_id=None, name='finalproduction', params=None, run_id='e6f5f1ea5b9d41a9a006806fd3f76896', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [ ]:
'''
# ignore below code

model_version = 2
prod_model_uri = f"models:/{production_model_name}@productionserver2"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]
'''

'\n# ignore below code\n\nmodel_version = 2\nprod_model_uri = f"models:/{production_model_name}@productionserver2"\n\nloaded_model = mlflow.xgboost.load_model(prod_model_uri)\ny_pred = loaded_model.predict(X_test)\ny_pred[:4]\n'